In [23]:
%reload_ext autoreload
%autoreload 2

%run init_imports.py

from photutils.isophote import Ellipse
from photutils.isophote import EllipseGeometry
from photutils.aperture import EllipticalAperture
from photutils.isophote import build_ellipse_model
plt.rcParams['font.family'] = 'monospace'

In [24]:
name = 'fit_sersic_exp_own_mask'

galaxy_name = "UGC09629"
imageFile = f"../data/{galaxy_name}_i.fits"
maskFile = f"../data/{galaxy_name}_own_mask.fits"
data_ = pyimfit.FixImage(fits.getdata(imageFile))
shape = data_.shape
print(shape)
data_mask = pyimfit.FixImage(fits.getdata(maskFile))  
fwhm, beta = 1.25136, 3.59000
data_moffat = pyimfit.moffat_psf(fwhm=fwhm, beta=beta, 
                PA=0.0, ell=0.0, size=31)
gain, sky, zcal, ron = 6.565, 221.61079, -23.59790, 5.76

# data_da = xr.DataArray(data)
# data_da = data_da.where(data_da >= 0, 1)
# data = data_da.values
cte = + np.abs(np.min(data_))*1.01
data = data_ + cte

(276, 231)
ModelObjectWrapper: about to call _model.CreateModelImage()...


In [28]:
x0, y0 = 111.0, 146.0
# Exp params
PA_exp, ell_exp, I_0, h = 38, 0.65, 497, 30.3575
# Sersic params
PA_sersic, ell_sersic, n, I_e, r_e = 18, 0.2, 1.35, 700, 5.77

model_desc = ut.exp_sersic_model(
    x0=x0, y0=y0, 
    PA_sersic=PA_sersic, ell_sersic=ell_sersic, n=n, I_e=I_e, r_e=r_e,
    PA_exp=PA_exp, ell_exp=ell_exp, I_0=I_0, h=h
    )

imfit_fitter = pyimfit.Imfit(model_desc, psf=data_moffat)
imfit_fitter.fit(data_, mask=data_mask, gain=gain, read_noise=ron, original_sky=sky)
data_fit = imfit_fitter.getModelImage()

savepath = f'../results/{name}_params.txt'
results = ut.get_dic_result(imfit_fitter, printit=True, savepath=savepath)

Parameter            Value                Error               
------------------------------------------------------------
X0_1                 111.9                0.0                 
Y0_1                 147.02               0.0                 
PA_1                 32.37                0.24                
ell_sersic_1         0.2                  0.0                 
n_1                  1.45                 0.01                
I_e_1                787.2                4.17                
r_e_1                5.97                 0.02                
PA_2                 29.23                0.06                
ell_exp_2            0.65                 0.0                 
I_0_2                303.89               1.77                
h_2                  29.73                0.11                
chi^2                2.48                
AIC                  37655.87            
BIC                  37739.76            


In [14]:
model_exp_desc = ut.exp_model(
    x0=results['X0_1'][0], y0=results['Y0_1'][0], 
    PA_exp=results['PA_2'][0], ell_exp=results['ell_exp_2'][0], I_0=results['I_0_2'][0], h=results['h_2'][0]
    )
data_fit_exp = ut.get_model_data(model_exp_desc, shape=shape)

model_sers_desc = ut.sersic_model(
    x0=results['X0_1'][0], y0=results['Y0_1'][0], 
    PA_sersic=results['PA_1'][0], ell_sersic=results['ell_sersic_1'][0], 
    n=results['n_1'][0], I_e=results['I_e_1'][0], r_e=results['r_e_1'][0]
    )
data_fit_sersic = ut.get_model_data(model_sers_desc, shape=shape)


ModelObjectWrapper: about to call _model.CreateModelImage()...
ModelObjectWrapper: about to call _model.CreateModelImage()...


In [16]:
x0, y0, sma, eps, pa = 111, 146, 10, 0.65, (38+90) * np.pi / 180
geometry = EllipseGeometry(x0=x0, y0=y0, sma=sma, eps=eps,
                           pa=pa)
ellipse = Ellipse(data, geometry)
isolist = ellipse.fit_image(sma0=0)

ellipse_fit = Ellipse(data_fit+cte, geometry)
isolist_fit = ellipse_fit.fit_image(sma0=0)

ellipse_fit_exp = Ellipse(data_fit_exp+cte, geometry)
isolist_fit_exp = ellipse_fit_exp.fit_image(sma0=0)

ellipse_fit_sersic = Ellipse(data_fit_sersic+cte, geometry)
isolist_fit_sersic = ellipse_fit_sersic.fit_image(sma0=0)

In [21]:
data_mag = ut.ADU_to_mag(data, sky=0, gain=gain, zcal=zcal)
data_fit_mag = ut.ADU_to_mag(data_fit, sky=cte, gain=gain, zcal=zcal)
data_res_mag = data_mag - data_fit_mag

m, n = data_mag.shape
coords_x = np.arange(m)
coords_y = np.arange(n)

data_mag_da = xr.DataArray(data_mag, coords=[('x', coords_x), ('y', coords_y)])
data_fit_mag_da = xr.DataArray(data_fit_mag, coords=[('x', coords_x), ('y', coords_y)])
data_res_mag_da = xr.DataArray(data_res_mag, coords=[('x', coords_x), ('y', coords_y)])
data_mask_da = xr.DataArray(data_mask, coords=[('x', coords_x), ('y', coords_y)])

ds = xr.Dataset({
    'data_mag': data_mag_da,
    'data_fit_mag': data_fit_mag_da,
    'data_res_mag': data_res_mag_da,
    'mask': data_mask_da
})


savepath = f'../results/{name}.nc'
ds.to_netcdf(savepath)

In [22]:
%reload_ext autoreload
%autoreload 2

import plot_utils as pu

res, rat = 1080, 1.5
fig, axs, fs, ax_in = pu.figure_skeleton(fig_w=rat*res, fig_h=res,
ratio=None, 
top_bool=True)

R, d, fact = 0.396, 115.3e6, 206265
xs = isolist.sma * R

profile_mag = ut.ADU_to_mag(isolist.intens, sky=0, gain=gain, zcal=zcal)
profile_err_mag = 2.5 * np.abs(isolist.int_err / (isolist.intens * np.log(10)))
pa = isolist.pa * 180 / np.pi - 90
pa_err = isolist.pa_err * 180 / np.pi 

dic = {
    '$\mu_r$ (mag/arcsec)': [profile_mag, profile_err_mag],
    '$\epsilon$': [isolist.eps, isolist.ellip_err],
    'PA (º)': [pa, pa_err],
}

for i in range(3):
    current_xticks = axs[i][1].get_xticks()
    new_xtick_labels = np.around(current_xticks * d / fact * 1e-3, decimals=1)

xs_fit = (isolist_fit.sma * R)
profile_fit_mag = ut.ADU_to_mag(isolist_fit.intens, sky=
0, gain=gain, zcal=zcal)

fit_ = profile_fit_mag[:len(isolist.sma)] 
xlim = [0, xs[-5]]
ylim = [7.9, 13.5]
pu.plot_val2([axs[0][1], axs[0][2]], xs, dic, '$\mu_r$ (mag/arcsec)', fs, 
            fit_, ylabel2='$\Delta \mu_r$', quot=False, xlim=xlim,
            ylim1=ylim,
            ylim2=[-0.3,0.3])

axs[0][1].plot(xs_fit, profile_fit_mag, c='r', ls='--',
lw=0.1*fs, zorder=2)

xs_fit_exp = isolist_fit_exp.sma * R
profile_fit_exp_mag = ut.ADU_to_mag(isolist_fit_exp.intens, sky=
0, gain=gain, zcal=zcal)
axs[0][1].plot(xs_fit_exp, profile_fit_exp_mag, c='b', ls='--',
lw=0.1*fs, zorder=2)

xs_fit_sersic = isolist_fit_sersic.sma * R
profile_fit_sersic_mag = ut.ADU_to_mag(isolist_fit_sersic.intens, sky=
0, gain=gain, zcal=zcal)
axs[0][1].plot(xs_fit_sersic, profile_fit_sersic_mag, c='g', ls='--',
lw=0.1*fs, zorder=2)
axs[0][1].invert_yaxis()

fit_eps = isolist_fit.eps[:len(isolist.sma)] 
pu.plot_val2([axs[1][1], axs[1][2]], xs, dic, '$\epsilon$', fs,
             fit_eps,
            ylabel2='$\Delta \epsilon / \epsilon$', xlim=xlim,
            ylim2=[-0.3,0.3])
axs[1][1].plot(xs_fit, isolist_fit.eps, c='r', ls='--',
lw=0.1*fs, zorder=2)

fit_pa = isolist_fit.pa[:len(isolist.sma)] * 180 / np.pi - 90
pu.plot_val2([axs[2][1], axs[2][2]], xs, dic, 'PA (º)', fs,
             fit_pa,
            ylabel2='$\Delta \\text{PA}$', xlim=xlim,
            ylim1=[0, 100],
            ylim2=[-20,20], quot=False)
axs[2][1].plot(xs_fit, isolist_fit.pa * 180 / np.pi - 90, c='r', ls='--',
lw=0.1*fs, zorder=2)




# ax_in.scatter(xs, profile_mag, c='k', s=0.015*fs, zorder=2, marker='s')
ax_in.plot(xs, profile_mag, c='k', lw=0.1*fs, zorder=1)
ax_in.plot(xs_fit, profile_fit_mag, c='r', ls='--',
lw=0.1*fs, zorder=2)
ax_in.plot(xs_fit_exp, profile_fit_exp_mag, c='b', ls='--',
lw=0.1*fs, zorder=2)
ax_in.plot(xs_fit_sersic, profile_fit_sersic_mag, c='g', ls='--',
lw=0.1*fs, zorder=2)
ax_in.set_xlim(0, 3.75)
ax_in.set_ylim(8, 9.75)
ax_in.invert_yaxis()

shrink, pad = 0.82, 0
vms = [8, 14.25]

pu.magplot(data_mag, axs[0][0], fs, vms=vms, shrink=shrink, pad=pad, mask=None)
pu.magplot(data_fit_mag, axs[1][0], fs, vms=vms, pad=pad, shrink=shrink)
vm_res = 1
pu.magplot(data_res_mag, axs[2][0], fs, pad=pad, shrink=shrink, vms=[-vm_res,vm_res], 
cmap='RdBu', mask=data_mask, chi=results['chi^2'])

smas = np.linspace(5, 120, 5)
gap = [7, 15,15,15,15]
for i, sma in enumerate(smas):
    iso = isolist.get_closest(sma)
    x, y = iso.sampled_coordinates()
    x, y = x*R, y*R

    ind = i*0
    break_at = int(0.5*len(x)) + ind  # For example, break at the quarter point
    
    color='k' if i < 3 else 'w'

    lw = 0.05
    axs[0][0].plot(x[:break_at-gap[i]//2], y[:break_at-gap[i]//2], color=color, lw=lw*fs)
    label_x = x[break_at]
    label_y = y[break_at]
    axs[0][0].text(label_x, label_y, f"{sma*R:.0f}", color=color, ha='center', va='center',
    fontsize=0.7*fs)

    axs[0][0].plot(x[break_at+gap[i]//2:], y[break_at+gap[i]//2:], color=color, lw=lw*fs)


labels = ['a)', 'b)', 'c)', 'd)', 'e)', 'f)', 'g)', 'h)', 'i)']
x_coords = np.linspace(0.32, 0.91, 3)
y_coords = [0.85, 0.445, 0.16]

for i, x in enumerate(x_coords):
    for j, y in enumerate(y_coords):
        index = 3*j + i
        fig.text(x, y, labels[index], fontsize=fs*1.25,
        fontweight='bold', color='darkred')


x_pos = 0.145
y_pos = np.linspace(0.41, 0.365, 4)
ff = 0.85
fig.text(x_pos, y_pos[0], 'Data', color='k', fontsize=ff*fs)
fig.text(x_pos, y_pos[1], 'Sersic+Exp', color='r', fontsize=ff*fs)
fig.text(x_pos, y_pos[2], 'Exp', color='b', fontsize=ff*fs)
fig.text(x_pos, y_pos[3], 'Sersic', color='g', fontsize=ff*fs)

savefold = '../figures/'
if not os.path.exists(savefold):
    os.makedirs(savefold)
filename = f'{name}.png'
savepath = os.path.join(savefold, filename)

# save figure
fig.savefig(savepath, dpi=300, bbox_inches='tight')

plt.close()

/home/prossello/extragalactic_lab1/code/plot_utils.py:397: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax0.set_xticklabels(new_xtick_labels)
/home/prossello/extragalactic_lab1/code/plot_utils.py:379: RuntimeWarning: invalid value encountered in divide
  err = vals_err_fit/vals
/home/prossello/extragalactic_lab1/code/plot_utils.py:397: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax0.set_xticklabels(new_xtick_labels)
/home/prossello/extragalactic_lab1/code/plot_utils.py:397: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax0.set_xticklabels(new_xtick_labels)


In [20]:
N = len(isolist.sma)
x = isolist.sma
dif = profile_mag - profile_fit_mag[:N]

with open(f"../results/{name}_dif.txt", "w") as f:
    f.write("x\tdif\n")  # optional header
    for xi, di in zip(x, dif):
        f.write(f"{xi}\t{di}\n")